In [7]:
#import libraries used into
from xml.etree.ElementTree import tostring
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [8]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
filepm = "OT" + datetime.now().strftime('%d.%m.%Y') + "5 P.M.XLSX"
main_path = "C:\\Users\\mruiz\\Documents\\DB OT" 
current_month = datetime.now().strftime('%B')
main_path_ = os.listdir(main_path)

In [9]:
for item in main_path_:
    if item == fileam:
        item1 = item
        print("Los datos del día de hoy son : ", item1)
        break

data1 = main_path + "\\" + item1
data1excel = pd.read_excel(data1, sheet_name="Sheet1", header=0)

Los datos del día de hoy son :  OT 13.10.2022.XLSX


In [10]:
data1excel = data1excel.assign(Material=0)

In [ ]:
data1excel = data1excel.drop('Material',axis=1)

Para convertir y eliminar el archivo creado desde to_csv usar el siguiente codigo: 
for item in main_path_:
    if item == file:
        item1 = item
        print("Los datos del día de hoy son : ", item1)
        break
"""try:
    os.remove(main_path + "\\daily_data.csv")
except OSError as e:
    print(f"Error:{ e.strerror}")"""

data1 = main_path + "\\" + item1
data1excel = pd.read_excel(data1, sheet_name="Sheet1", header=0)
data1excel_ = pd.DataFrame(data1excel)
"""path_csv = main_path + "\\daily_data.csv"
data1csv = data1excel_.to_csv(path_csv)
data1csv_ = pd.read_csv(path_csv)"""

In [ ]:
data1excel['Material'] = np.where((data1excel.Clase_de_orden =='ZMCP') & (data1excel.Texto_breve.__contains__('EMERG' or 'REVI')),'CONTIENE MATERIAL',"DA")

In [ ]:
print(data1excel.head())
print(data1excel.columns)

In [11]:
for item in main_path_:
    if item == 'Master_OT.xlsx':
        item2 = item
        print("La base de datos de nombres es : ", item2)
        break

data2 = main_path + "\\" + item2
data2excel = pd.read_excel(data2, sheet_name="Names", header=0)

La base de datos de nombres es :  Master_OT.xlsx


In [12]:
data3 = main_path + "\\POLITICA.XLSX"
data3excel = pd.read_excel(data3, sheet_name="POLITICAS", header=0)

In [ ]:
print(data2excel.head())

In [ ]:
result_df = pd.resul

In [ ]:
result_df = pd.merge(data1excel,data2excel,left_on="Pto_tbjo_responsable",right_on="TecID",how="left")

In [ ]:
result_df = pd.merge(result_df,data3excel,on="Equipo",how="left")

In [ ]:
print(result_df.loc[0])

In [ ]:
print(result_df.columns)
print(data1excel.shape[0])
print(result_df.shape[0])
print(result_df.head())


In [ ]:
os.remove(main_path + "\\main.xlsx")
path = main_path + "\\main.xlsx"
result_df.to_excel(path, index=False,header=True)